# Adversarial Attack

## Data Preparation and Initialization

### Import Packages

In [ ]:
import torch
import torch.nn as nn

import os
import glob
import shutil
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader

from pytorchcv.model_provider import get_model as ptcv_get_model

### Download Data

In [ ]:
!pip install pytorchcv

In [ ]:
!gdown '19E0B_Cj2gCWSHiqI6wkFaHYXXXGVA0WR' -O data.zip
!unzip ./data.zip
!rm ./data.zip

### Initialization

* $\epsilon$ is fixed to be 8. But on **Data section**, we will first apply transforms on raw pixel value (0-255 scale) **by ToTensor (to 0-1 scale)** and then **Normalize (subtract mean divide std)**. $\epsilon$ should be set to $\frac{8}{255 * std}$ during attack.

* Explaination (optional)
    * Denote the first pixel of original image as $p$, and the first pixel of adversarial image as $a$.
    * The $\epsilon$ constraints tell us $\left| p-a \right| <= 8$.
    * ToTensor() can be seen as a function where $T(x) = x/255$.
    * Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.
    * After applying ToTensor() and Normalize() on $p$ and $a$, the constraint becomes $\left| N(T(p))-N(T(a)) \right| = \left| \frac{\frac{p}{255}-mean}{std}-\frac{\frac{a}{255}-mean}{std} \right| = \frac{1}{255 * std} \left| p-a \right| <= \frac{8}{255 * std}.$
    * So, we should set $\epsilon$ to be $\frac{8}{255 * std}$ after ToTensor() and Normalize().

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 8

# the mean and std are the calculated statistics from cifar_10 dataset
cifar_10_mean = (0.491, 0.482, 0.447) # mean for the three channels of cifar_10 images
cifar_10_std = (0.202, 0.199, 0.201) # std for the three channels of cifar_10 images

# convert mean and std to 3-dimensional tensors for future operations
mean = torch.tensor(cifar_10_mean).to(device).view(3, 1, 1)
std = torch.tensor(cifar_10_std).to(device).view(3, 1, 1)

epsilon = 8/255/std

# Step size
alpha = 0.8/255/std

root = './data' # directory for storing benign images

### Data Transformation

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar_10_mean, cifar_10_std)
])

### Dataset

In [ ]:
class AdvDataset(Dataset):
    def __init__(self, data_dir, transform):
        self.images = []
        self.labels = []
        self.names = []
        '''
        data_dir
        ├── class_dir
        │   ├── class1.png
        │   ├── ...
        │   ├── class20.png
        '''
        for i, class_dir in enumerate(sorted(glob.glob(f'{data_dir}/*'))):
            images = sorted(glob.glob(f'{class_dir}/*'))
            self.images += images
            self.labels += ([i] * len(images))
            self.names += [os.path.relpath(imgs, data_dir) for imgs in images]
        self.transform = transform

    def __getitem__(self, idx):
        image = self.transform(Image.open(self.images[idx]))
        label = self.labels[idx]
        return image, label
    
    def __getname__(self):
        return self.names
        
    def __len__(self):
        return len(self.images)

## Model Training

### Construct Dataset and Dataloader

In [ ]:
adv_set = AdvDataset(root, transform=transform)
adv_names = adv_set.__getname__()
adv_loader = DataLoader(adv_set, batch_size=batch_size, shuffle=False)

### Helper Function

In [ ]:
# Evaluate the performance of model on benign images
def epoch_benign(model, loader, loss_fn):
    model.eval()
    train_acc, train_loss = 0.0, 0.0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        yp = model(x)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
    return train_acc / len(loader.dataset), train_loss / len(loader.dataset)


### FGSM Attack

In [ ]:
def fgsm(modellist, x, y, loss_fn, epsilon=epsilon):
    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
    loss = 0
    for i in range(len(modellist)):
        modellist[i].eval()
        loss += loss_fn(modellist[i](x_adv), y) # calculate loss
    loss.backward() # calculate gradient
    # fgsm: use gradient ascent on x_adv to maximize loss
    x_adv = x_adv + epsilon * x_adv.grad.detach().sign()
    
    return x_adv

### Iterative FGSM Attack

In [ ]:
def ifgsm(modellist, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=1000):
    '''
    initialize x_adv as original benign image x
    write a loop of num_iter to represent the iterative times
    for each loop
        call fgsm with (epsilon = alpha) to obtain new x_adv
        clip new x_adv back to [x-epsilon, x+epsilon]
    '''
    x_adv = x.detach().clone()
    for i in range(num_iter):
        x_adv = fgsm(modellist, x_adv, y, loss_fn, alpha)
        x_adv = torch.max(torch.min(x_adv, x+epsilon), x-epsilon)
    
    return x_adv

## Attack Function

- Recall
    - ToTensor() can be seen as a function where $T(x) = x/255$.
    - Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.
    
- Inverse function
    - Inverse Normalize() can be seen as a function where $N^{-1}(x) = x*std+mean$ where $mean$ and $std$ are constants.
    - Inverse ToTensor() can be seen as a function where $T^{-1}(x) = x*255$.
  
- Special Noted
    - ToTensor() will also convert the image from shape (height, width, channel) to shape (channel, height, width), so we also need to transpose the shape back to original shape.
    - Since our dataloader samples a batch of data, what we need here is to transpose **(batch_size, channel, height, width)** back to **(batch_size, height, width, channel)** using np.transpose.

In [ ]:
# perform adversarial attack and generate adversarial examples
def gen_adv_examples(modellist, loader, attack, loss_fn):
    train_acc_list, train_loss_list = [0.0]*len(modellist), [0.0]*len(modellist)
    for i, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        x_adv = attack(modellist, x, y, loss_fn) # obtain adversarial examples
        for j in range(len(modellist)):
            modellist[j].eval()
            yp = modellist[j](x_adv)
            loss = loss_fn(yp, y)
            train_acc_list[j] += (yp.argmax(dim=1) == y).sum().item()
            train_loss_list[j] += loss.item() * x.shape[0]
        # store adversarial examples
        adv_ex = ((x_adv) * std + mean).clamp(0, 1) # to 0-1 scale
        adv_ex = (adv_ex * 255).clamp(0, 255) # 0-255 scale
        adv_ex = adv_ex.detach().cpu().data.numpy().round() # round to remove decimal part
        adv_ex = adv_ex.transpose((0, 2, 3, 1)) # transpose (bs, C, H, W) back to (bs, H, W, C)
        advs = adv_ex if i == 0 else np.r_[advs, adv_ex]

    return advs, train_acc_list, train_loss_list

In [ ]:
# create directory which stores adversarial examples
def create_dir(data_dir, adv_dir, adv_examples, adv_names):
    if os.path.exists(adv_dir) is not True:
        _ = shutil.copytree(data_dir, adv_dir)
    for example, name in zip(adv_examples, adv_names):
        im = Image.fromarray(example.astype(np.uint8)) # image pixel value should be unsigned int
        im.save(os.path.join(adv_dir, name))

### Backbone Model Construction

In [ ]:
model_1 = ptcv_get_model('resnet20_cifar10', pretrained=True).to(device)
model_2 = ptcv_get_model('preresnet20_cifar10', pretrained=True).to(device)
model_3 = ptcv_get_model('pyramidnet110_a48_cifar10', pretrained=True).to(device)
model_4 = ptcv_get_model('seresnet20_cifar10', pretrained=True).to(device)
model_5 = ptcv_get_model('diaresnet20_cifar10', pretrained=True).to(device)
model_6 = ptcv_get_model('diapreresnet20_cifar10', pretrained=True).to(device)
model_7 = ptcv_get_model('densenet40_k12_cifar10', pretrained=True).to(device)
model_8 = ptcv_get_model('wrn16_10_cifar10', pretrained=True).to(device)
model_9 = ptcv_get_model('nin_cifar10', pretrained=True).to(device)
model_10 = ptcv_get_model('shakeshakeresnet20_2x16d_cifar10', pretrained=True).to(device)
model_11 = ptcv_get_model('sepreresnet20_cifar10', pretrained=True).to(device)
model_12 = ptcv_get_model('xdensenet40_2_k24_bc_cifar10', pretrained=True).to(device)
model_13 = ptcv_get_model('ror3_56_cifar10', pretrained=True).to(device)
model_14 = ptcv_get_model('rir_cifar10', pretrained=True).to(device)

model_list = []
model_list.append(model_1)
model_list.append(model_2)
model_list.append(model_3)
model_list.append(model_4)
model_list.append(model_5)
model_list.append(model_6)
model_list.append(model_7)
model_list.append(model_8)
model_list.append(model_9)
model_list.append(model_10)
model_list.append(model_11)
model_list.append(model_12)
model_list.append(model_13)
model_list.append(model_14)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

for i in range(len(model_list)):
    benign_acc, benign_loss = epoch_benign(model_list[i], adv_loader, loss_fn)
    print(f'num: {i}, benign_acc = {benign_acc:.5f}, benign_loss = {benign_loss:.5f}')


### FGSM Attack

In [ ]:
# model = model_1
# adv_examples, fgsm_acc, fgsm_loss = gen_adv_examples(model, adv_loader, fgsm, loss_fn)
# print(f'fgsm_acc = {fgsm_acc:.5f}, fgsm_loss = {fgsm_loss:.5f}')

# create_dir(root, 'fgsm', adv_examples, adv_names)

### I-FGSM Attack

In [ ]:
adv_examples, ifgsm_acc, ifgsm_loss = gen_adv_examples(model_list, adv_loader, ifgsm, loss_fn)
for i in range(len(model_list)):
    print(f'num: {i}, ifgsm_acc = {ifgsm_acc[i]/len(adv_loader.dataset):.5f}, ifgsm_loss = {ifgsm_loss[i]/len(adv_loader.dataset):.5f}')

create_dir(root, 'en_ifgsm', adv_examples, adv_names)

## Compress the images

In [ ]:
# %cd fgsm
# !tar zcvf ../fgsm.tgz *
# %cd ..

%cd ifgsm
!tar zcvf ../ifgsm.tgz *
%cd ..

## Visualization

In [ ]:
model = model_1 # choose a model to visualize the performance

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10, 20))
cnt = 0
for i, cls_name in enumerate(classes):
    path = f'{cls_name}/{cls_name}1.png'
    # benign image
    cnt += 1
    plt.subplot(len(classes), 4, cnt)
    im = Image.open(f'./data/{path}')
    logit = model(transform(im).unsqueeze(0).to(device))[0]
    predict = logit.argmax(-1).item()
    prob = logit.softmax(-1)[predict].item()
    plt.title(f'benign: {cls_name}1.png\n{classes[predict]}: {prob:.2%}')
    plt.axis('off')
    plt.imshow(np.array(im))
    # adversarial image
    cnt += 1
    plt.subplot(len(classes), 4, cnt)
    im = Image.open(f'./fgsm/{path}')
    logit = model(transform(im).unsqueeze(0).to(device))[0]
    predict = logit.argmax(-1).item()
    prob = logit.softmax(-1)[predict].item()
    plt.title(f'adversarial: {cls_name}1.png\n{classes[predict]}: {prob:.2%}')
    plt.axis('off')
    plt.imshow(np.array(im))
plt.tight_layout()
plt.show()